In [ ]:
import logging
import os
import datetime 
import time
import requests
import urllib.request
from lxml import etree
from fake_useragent import UserAgent
import pandas as pd 
import gzip
import shutil 
import sys
import psycopg2

In [ ]:
baseurl='https://www.realestate.com.au/xml-sitemap/'
PageSaveFolder=r'D:\shared_folder\airflow_logs\XML_save_folder\gz_files\\' #'/opt/airflow/logs/XML_save_folder/raw_sitemap'
ScrapeFile = 'pdp-sitemap-buy-1.xml.gz'

In [37]:
print('gz file:', ScrapeFile)
#download from sitemap, use dynamic variable 
sitemap_url = baseurl #'https://www.realestate.com.au/xml-sitemap/'#pdp-sitemap-buy-1.xml.gz' 
_file=ScrapeFile #im lazy, sue me
gz_save_name =_file[:-7] + '_' + (datetime.datetime.now()).strftime('%Y-%m-%d') + '.gz'
gz_url = sitemap_url + _file
gz_save_path = PageSaveFolder

gz file: pdp-sitemap-buy-1.xml.gz


In [39]:
urllib.request.urlretrieve(gz_url, gz_save_path + gz_save_name)

('D:\\shared_folder\\airflow_logs\\XML_save_folder\\gz_files\\\\pdp-sitemap-buy-1_2021-01-01.gz',
 <http.client.HTTPMessage at 0x27888caefc8>)

In [38]:
#save gz to dir for archiving 
print("file:", gz_save_name)
print("written to dir:", gz_save_path + gz_save_name)

file: pdp-sitemap-buy-1_2021-01-01.gz
written to dir: D:\shared_folder\airflow_logs\XML_save_folder\gz_files\\pdp-sitemap-buy-1_2021-01-01.gz


In [40]:
#feast upon that rich gooey xml 
_xml_save = _file[:-7] + '_' + (datetime.datetime.now()).strftime('%Y-%m-%d') + '.xml'  
with gzip.open(gz_save_path + gz_save_name, 'rb') as f_in:
    with open(gz_save_path + _xml_save, 'wb') as f_out: 
        shutil.copyfileobj(f_in, f_out)

In [ ]:

#xml part 
root = etree.parse(gz_save_path + _xml_save)
XML_gz_Dataset=pd.DataFrame(columns =['Parent_gz','ScrapeDT','Url', 'PropType', 'State', 'Suburb', 'PropID', 'LastMod', 'ExternalIP'])
_PropType=_State=_PropID=_LastMod=_split=_Url=""
_external_ip = urllib.request.urlopen('https://ident.me').read().decode('utf8')
#iterate xml
for element in root.iter():
    #writes results to df, same as the previous module 
    if 'url' in element.tag and _Url != '':
        XML_gz_Dataset=XML_gz_Dataset.append({
                    'Parent_gz': str(ScrapeFile)
                    ,'ScrapeDT' : (datetime.datetime.now()).strftime('%Y-%m-%d %H:%M:%S')
                    , 'Url' : str(_Url)
                    , 'PropType' : str(_PropType)
                    , 'State' : str(_State)
                    , 'Suburb' : str(_Suburb)
                    , 'PropID' : str(_PropID)
                    , 'LastMod': str(_LastMod)
                    , 'ExternalIP': str(_external_ip)
                    } ,ignore_index=True) 
        _PropType=_State=_PropID=_LastMod=_split=_Url=""
    if 'lastmod' in element.tag: 
        _LastMod = element.text
    #just about everything gleaned from loac (url) tag
    elif 'loc' in element.tag: 
        if '-tas-' in element.text: 
            _State='tas'
        elif '-vic-' in element.text: 
            _State='vic'
        elif '-nsw-' in element.text: 
            _State='nsw'
        elif '-act-' in element.text: 
            _State='act'
        elif '-qld-' in element.text: 
            _State='qld'
        elif '-nt-' in element.text: 
            _State='nt'
        elif '-sa-' in element.text: 
            _State='sa'
        elif '-wa-' in element.text: 
            _State='wa'
        
        _Url = element.text
        _split=str(element.text).split(_State)
        #had to do it this way so unconventional suburb names are still caught
        _PropType = _split[0].replace('https://www.realestate.com.au/property-','')[:-1]
        _split=str(element.text).split('-')
        _Suburb=_split[len(_split) -2 ]
        _PropID=_split[len(_split) -1 ]

XML_gz_Dataset.to_csv(gz_save_path + _xml_save[:-3] + '_results' +'.csv')
print("file saved to: " + gz_save_path + _xml_save[:-3] + '_results' +'.csv')

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#io-sql-method
import csv
from io import StringIO

from sqlalchemy import create_engine

def psql_insert_copy(table, conn, keys, data_iter):
    """
    Execute SQL statement inserting data

    Parameters
    ----------
    table : pandas.io.sql.SQLTable
    conn : sqlalchemy.engine.Engine or sqlalchemy.engine.Connection
    keys : list of str
        Column names
    data_iter : Iterable that iterates the values to be inserted
    """
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)